<a href="https://colab.research.google.com/github/luciosjodin/ProcesamientoDelHabla/blob/master/Sj%C3%B6din_TP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP4 Chatbots

# Chatbots basados en recuperación

En inglés information retrieval chatbots

## Motor de búsqueda

* Búsqueda por palabras clave: Extrae palabras clave de la pregunta del usuario y busca coincidencias en las preguntas almacenadas.

* Similitud del coseno: Si has representado las preguntas como vectores (por ejemplo, usando TF-IDF o word embeddings), puedes usar la similitud del coseno para medir la distancia entre las preguntas.

* Word embeddings: Utiliza modelos de word embeddings como Word2Vec o BERT para obtener representaciones semánticas de las preguntas y las consultas del usuario.

### Búsqueda por palabras claves

In [1]:
tu_diccionario = {
   "hola": "¡Hola! ¿En qué puedo ayudarte?",
   "adiós": "Hasta luego. ¡Que tengas un buen día!",
   "información": "¿Qué tipo de información estás buscando?",
   # Agrega más entradas de diccionario según tus necesidades
}


In [2]:
def responder_pregunta(pregunta):
    pregunta_procesada = nlp(pregunta.lower())  # Procesa la pregunta y convierte a minúsculas
    respuesta = "Lo siento, no entiendo tu pregunta."

    # Busca una coincidencia en el diccionario
    for palabra in pregunta_procesada:
        # regresa la primer coincidencia que encuentra
        if palabra.text in tu_diccionario:
            respuesta = tu_diccionario[palabra.text]
            break

    return respuesta


In [3]:
"""
while True:
    entrada_usuario = input("Tú: ")
    if entrada_usuario.lower() == "salir":
        print("Chatbot: Hasta luego.")
        break
    respuesta = responder_pregunta(entrada_usuario)
    print("Chatbot:", respuesta)
"""

'\nwhile True:\n    entrada_usuario = input("Tú: ")\n    if entrada_usuario.lower() == "salir":\n        print("Chatbot: Hasta luego.")\n        break\n    respuesta = responder_pregunta(entrada_usuario)\n    print("Chatbot:", respuesta)\n'

Puede modificar la implementación anterior para evitar el bucle while True y usar un dataset de prueba de preguntas y respuestas.

## Búsqueda por similitud

Para los chatbots basados ​​en recuperación, es común utilizar bolsas de palabras (bag of words) o tf-idf para calcular la similitud de intenciones.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Datos de ejemplo
preguntas = ["¿Qué es el aprendizaje automático?",
             "¿Cómo funciona la regresión lineal?"]
respuestas = ["El aprendizaje automático es una rama de la inteligencia artificial...",
              "La regresión lineal es un método de modelado..."]

# Vectorización con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(preguntas)

# Función para encontrar la mejor coincidencia
def responder_pregunta(consulta_usuario):
    consulta_vec = vectorizer.transform([consulta_usuario])
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    print(similitudes)
    indice_mejor_coincidencia = similitudes.argmax()
    print(indice_mejor_coincidencia)
    return respuestas[indice_mejor_coincidencia]


In [5]:

# Ejemplo de consulta
consulta = "¿Qué es la regresión lineal?"
print(responder_pregunta(consulta))


[0.4 0.6]
1
La regresión lineal es un método de modelado...


## Búsqueda por similitud en embeddings

Puedes vectorizar el texto usando embeddings, de spacy por ejemplo, como vimos en clases.


In [6]:
#!pip install spacy --quiet
#!python -m spacy download es_core_news_sm --quiet

## Actividades



### 1) Elaborar un dataset de preguntas y respuestas para crear un Chatbot para un aplicación particular. ( 3 puntos )

1.1 Debe definir la aplicación (atención al cliente bancario, atención a estudiantes universitarios, etc).

1.2 El listado de preguntas y respuestas debe tener como mínimo 20 elementos pregunta - respuesta.

###  2) Crear el chatbot utilizando TFIDF y similitud del coseno. (1 punto)

### 3) Crear otro chatbot utilizando embeddings. Indique cuál embedding (1 punto) pre-entrenado eligió.

### 4) Muestra ambos chatbots funcionando (1 punto)

Adjuntar la lista de preguntas y respuestas utilizadas para probar el funcionamiento.

Releve o indique cuáles respondió correctamente y cuáles no.

### 5) Añade tus conclusiones de todo lo realizado (2 punto)

* Resalte e indique en cuáles respuestas falla o tiene problemas.
* Cuáles preguntas confunde.
* Compare los resultados de los chatbots.



### No olvides:

* Explicar tus decisiones y configuraciones. Añadir tus conclusiones.
* Anunciar en el foro cuál será tu aplicación y postear tu entrega y tus avances.
* Debes subir tu notebook a un repo GitHub público de tu propiedad compartido + enlace colab.
* Documentar todo el proceso.
* Citar tus fuentes





## 1.1 y 1.2)


###Creo el dataset con dos listas, 25 preguntas y sus 25 respuestas.
El Dataset creado pertenece imaginariamente al sitio web de una agencia inmobiliaria.

In [7]:
# Primero de todo, la importación de todas las librerías necesarias
!pip install spacy --quiet
!python -m spacy download es_core_news_md --quiet
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import numpy as np


✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
preguntas_inmobiliaria = [
    "¿Cómo puedo buscar propiedades en venta?",
    "¿Cuáles son los requisitos para alquilar una propiedad?",
    "¿Qué documentos necesito para vender mi propiedad?",
    "¿Ofrecen servicios de tasación de propiedades?",
    "¿Cómo puedo contactar a un agente inmobiliario?",
    "¿Tienen propiedades en una zona específica?",
    "¿Puedo ver fotos y videos de las propiedades?",
    "¿Cómo programo una visita a una propiedad?",
    "¿Qué es una hipoteca y cómo funciona?",
    "¿Cuál es la diferencia entre alquiler y venta?",
    "¿Puedo negociar el precio de una propiedad?",
    "¿Qué gastos adicionales debo considerar al comprar?",
    "¿Cómo funciona el proceso de escrituración?",
    "¿Ofrecen asesoramiento legal?",
    "¿Puedo vender mi propiedad si aún tengo hipoteca?",
    "¿Qué es el seguro de hogar y es obligatorio?",
    "¿Cómo puedo saber si una propiedad está libre de cargas?",
    "¿Cuál es el tiempo promedio para vender una propiedad?",
    "¿Qué servicios de postventa ofrecen?",
    "¿Puedo listar mi propiedad en su sitio web?",
    "¿Cómo manejan la privacidad de mis datos?",
    "¿Qué hago si tengo un problema con mi alquiler?",
    "¿Ofrecen opciones de financiamiento?",
    "¿Puedo alquilar una propiedad amueblada?",
    "¿Cómo puedo dar de baja mi propiedad del listado?"
]

respuestas_inmobiliaria = [
    "Puede buscar propiedades en venta utilizando nuestro buscador en la página principal. Puede filtrar por ubicación, tipo de propiedad, precio, etc.",
    "Los requisitos para alquilar varían según la propiedad y el propietario. Generalmente, se solicita comprobante de ingresos, garantía y referencias. Puede ver los requisitos específicos en la descripción de cada propiedad.",
    "Los documentos necesarios para vender su propiedad incluyen el título de propiedad, DNI de los titulares, planos, impuestos al día, entre otros. Le recomendamos contactar a uno de nuestros agentes para una lista completa.",
    "Sí, ofrecemos servicios de tasación de propiedades. Nuestros tasadores profesionales le brindarán un valor estimado basado en el mercado actual y las características de su propiedad.",
    "Puede contactar a un agente inmobiliario a través del formulario de contacto en nuestro sitio web, por teléfono o visitando nuestras oficinas.",
    "Sí, en nuestro buscador puede especificar la zona en la que está interesado y le mostraremos las propiedades disponibles en esa área.",
    "Sí, todas nuestras propiedades listadas incluyen fotos y, en muchos casos, videos o tours virtuales para que pueda tener una mejor idea de la propiedad.",
    "Para programar una visita a una propiedad, puede hacer clic en el botón 'Solicitar visita' en la página de la propiedad o contactarnos directamente.",
    "Una hipoteca es un préstamo que se utiliza para comprar una propiedad, donde la propiedad misma sirve como garantía. El funcionamiento implica pagos periódicos de capital e intereses.",
    "La principal diferencia es que en el alquiler se paga por el uso temporal de la propiedad, mientras que en la venta se adquiere la propiedad total de la misma.",
    "La posibilidad de negociar el precio depende del propietario y las condiciones del mercado. Nuestros agentes pueden asesorarlo sobre la viabilidad de una oferta.",
    "Al comprar una propiedad, debe considerar gastos adicionales como impuestos, honorarios notariales, gastos de registro, mudanza, etc.",
    "El proceso de escrituración es el acto legal en el que se transfiere la propiedad del vendedor al comprador ante un notario público.",
    "No ofrecemos asesoramiento legal directo, pero podemos recomendarle abogados especializados en derecho inmobiliario.",
    "Sí, es posible vender una propiedad con hipoteca. El proceso implica cancelar la hipoteca pendiente con los fondos de la venta.",
    "El seguro de hogar cubre daños a la propiedad y su contenido. En muchos casos, es obligatorio si tiene una hipoteca.",
    "Podemos ayudarlo a verificar si una propiedad tiene cargas (como hipotecas o embargos) solicitando un informe de dominio.",
    "El tiempo promedio para vender una propiedad varía según el mercado, el precio y las características de la propiedad. Nuestros agentes pueden darle una estimación más precisa.",
    "Nuestros servicios de postventa incluyen asesoramiento en trámites posteriores a la compra/venta y recomendaciones de servicios relacionados.",
    "Sí, puede listar su propiedad en nuestro sitio web. Contáctenos para conocer los requisitos y el proceso.",
    "Manejamos la privacidad de sus datos de acuerdo con nuestra política de privacidad, cumpliendo con las normativas vigentes.",
    "Si tiene un problema con su alquiler, le recomendamos contactar primero a su propietario o administrador. Si el problema persiste, podemos brindarle asesoramiento.",
    "Podemos ponerlo en contacto con instituciones financieras que ofrecen opciones de financiamiento para la compra de propiedades.",
    "Sí, algunas de nuestras propiedades en alquiler están amuebladas. Puede usar los filtros de búsqueda para encontrar propiedades amuebladas.",
    "Para dar de baja su propiedad del listado, por favor contáctenos indicando los detalles de la propiedad y su solicitud."
]

Creo una función para limpiar y normalizar las listas y las dejo en bloque aparte por si se quisiera probarel rendimiento sin este paso

In [9]:
def normalizar_texto(texto):
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar caracteres especiales, dejando solo letras, números y espacios
    texto = re.sub(r'[^a-z0-9\s]', '', texto)
    # Eliminar espacios extra y saltos de línea
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

Procedo a limpiar y normalizar la lista de preguntas del dataset.

In [10]:
# Limpiar y normalizar la lista de preguntas
preguntas_inmobiliaria = [normalizar_texto(pregunta) for pregunta in preguntas_inmobiliaria]

## 2)

### Creo el chatbot con TF-IDF y similitud del coseno

In [11]:
# Usamos los datasets de preguntas y respuestas de la inmobiliaria definidos anteriormente
preguntas = preguntas_inmobiliaria
respuestas = respuestas_inmobiliaria

# Inicializar el vectorizador TF-IDF
vectorizer = TfidfVectorizer()

# Ajustar y transformar las preguntas en una matriz TF-IDF
tfidf_matrix = vectorizer.fit_transform(preguntas_inmobiliaria)

# Función para encontrar la mejor coincidencia usando similitud del coseno
def responder_pregunta_tfidf(consulta_usuario):
    # Limpiar y normalizar la consulta
    consulta_limpia = normalizar_texto(consulta_usuario)

    # Transformar la consulta del usuario a vector TF-IDF
    consulta_vec = vectorizer.transform([consulta_limpia])

    # Calcular la similitud del coseno entre la consulta y todas las preguntas del dataset
    similitudes = cosine_similarity(consulta_vec, tfidf_matrix).flatten()

    # Encontrar el índice de la pregunta con la mayor similitud
    indice_mejor_coincidencia = similitudes.argmax()

    # Devolver la respuesta correspondiente a la pregunta con mayor similitud
    return respuestas[indice_mejor_coincidencia]

## 3)

### Creo el chatbot con embedding (spacy)

In [12]:
# Cargar el modelo de spaCy con embeddings
nlp = spacy.load("es_core_news_md")

preguntas_vec = [nlp(pregunta).vector for pregunta in preguntas_inmobiliaria]
# Convertimos la lista de vectores a una matriz NumPy para eficiencia
matriz_preguntas_vec = np.array(preguntas_vec)

# Función para encontrar la mejor coincidencia usando similitud de embeddings
def responder_pregunta_embeddings(consulta_usuario):
# Limpiar y normalizar la consulta
  consulta_limpia = normalizar_texto(consulta_usuario)
# Obtener el embedding de la consulta del usuario
  consulta_doc = nlp(consulta_limpia)
  consulta_vec = consulta_doc.vector
# Calcular la similitud del coseno entre la consulta y los vectores de las preguntas
# Es necesario reformar los vectores para que cosine_similarity funcione correctamente
  similitudes = cosine_similarity([consulta_vec], preguntas_vec).flatten()
#Encontrar el índice de la pregunta con la mayor similitud
  indice_mejor_coincidencia = similitudes.argmax()
# Devolver la respuesta correspondiente a la pregunta con mayor similitud
  return respuestas[indice_mejor_coincidencia]

**DEADLINE ENTREGA HASTA PUNTO 5 PARA REGULARIZAR O PROMOCIÓN** DOMINGO 09/11

## 4)

### Se crea un dataset con preguntas de prueba para mostrar la ejecución de ambos chatbots y comparar sus resultados.

In [13]:
# Lista de preguntas de prueba
preguntas_prueba = [
    "Quiero buscar casas en venta.",
    "Qué necesito para alquilar un departamento?",
    "¿Qué papeles requiero para vender mi casa?",
    "¿Hacen tasaciones de propiedades?",
    "¿Hola. Cómo hablo con un agente?",
    "¿Tienen propiedades en Palermo?",
    "¿Puedo ver fotos de las propiedades?",
    "¿Cómo pido una visita?",
    "Explícame qué es una hipoteca.",
    "¿Cuál es la diferencia entre alquilar y comprar?"
]

# Lista de respuestas esperadas correspondientes a las preguntas de prueba
respuestas_esperadas = [
    "Puede buscar propiedades en venta utilizando nuestro buscador en la página principal. Puede filtrar por ubicación, tipo de propiedad, precio, etc.",
    "Los requisitos para alquilar varían según la propiedad y el propietario. Generalmente, se solicita comprobante de ingresos, garantía y referencias. Puede ver los requisitos específicos en la descripción de cada propiedad.",
    "Los documentos necesarios para vender su propiedad incluyen el título de propiedad, DNI de los titulares, planos, impuestos al día, entre otros. Le recomendamos contactar a uno de nuestros agentes para una lista completa.",
    "Sí, ofrecemos servicios de tasación de propiedades. Nuestros tasadores profesionales le brindarán un valor estimado basado en el mercado actual y las características de su propiedad.",
    "Puede contactar a un agente inmobiliario a través del formulario de contacto en nuestro sitio web, por teléfono o visitando nuestras oficinas.",
    "Sí, en nuestro buscador puede especificar la zona en la que está interesado y le mostraremos las propiedades disponibles en esa área.",
    "Sí, todas nuestras propiedades listadas incluyen fotos y, en muchos casos, videos o tours virtuales para que pueda tener una mejor idea de la propiedad.",
    "Para programar una visita a una propiedad, puede hacer clic en el botón 'Solicitar visita' en la página de la propiedad o contactarnos directamente.",
    "Una hipoteca es un préstamo que se utiliza para comprar una propiedad, donde la propiedad misma sirve como garantía. El funcionamiento implica pagos periódicos de capital e intereses.",
    "La principal diferencia es que en el alquiler se paga por el uso temporal de la propiedad, mientras que en la venta se adquiere la propiedad total de la misma."
]


No se procede a limpiar y normalizar las lista de preguntas del dataset de prueba ya que la función de limpieza está dentro de la ejecución de los chatbots

### Chatbot TF-IDF

Se ejecuta la función `responder_pregunta_tfidf` con el dataset de prueba y se almacenan los resultados.


In [14]:
# Inicializar una lista vacía para almacenar las respuestas del chatbot TF-IDF
resultados_tfidf = []

# Iterar a través de cada pregunta de prueba y obtener la respuesta del chatbot TF-IDF
for pregunta in preguntas_prueba:
    respuesta = responder_pregunta_tfidf(pregunta)
    resultados_tfidf.append(respuesta)

Se itera a través de las preguntas de prueba, se obtienen las respuestas del chatbot TF-IDF y las respuestas esperadas, se comparan y se imprimen los resultados de forma prolija (Correcto o Incorrecto)



In [15]:
# Imprimir los resultados del chatbot TF-IDF
print("--- Resultados del Chatbot TF-IDF ---")
for i in range(len(preguntas_prueba)):
    pregunta = preguntas_prueba[i]
    respuesta_tfidf = resultados_tfidf[i]
    respuesta_esperada = respuestas_esperadas[i]

    es_correcta = "Correcta" if respuesta_tfidf == respuesta_esperada else "Incorrecta"

    print(f"\nPregunta: {pregunta}")
    print(f"Respuesta Chatbot (TF-IDF): {respuesta_tfidf}")
    print(f"Respuesta Esperada: {respuesta_esperada}")
    print(f"Resultado: {es_correcta}")


--- Resultados del Chatbot TF-IDF ---

Pregunta: Quiero buscar casas en venta.
Respuesta Chatbot (TF-IDF): Puede buscar propiedades en venta utilizando nuestro buscador en la página principal. Puede filtrar por ubicación, tipo de propiedad, precio, etc.
Respuesta Esperada: Puede buscar propiedades en venta utilizando nuestro buscador en la página principal. Puede filtrar por ubicación, tipo de propiedad, precio, etc.
Resultado: Correcta

Pregunta: Qué necesito para alquilar un departamento?
Respuesta Chatbot (TF-IDF): Los documentos necesarios para vender su propiedad incluyen el título de propiedad, DNI de los titulares, planos, impuestos al día, entre otros. Le recomendamos contactar a uno de nuestros agentes para una lista completa.
Respuesta Esperada: Los requisitos para alquilar varían según la propiedad y el propietario. Generalmente, se solicita comprobante de ingresos, garantía y referencias. Puede ver los requisitos específicos en la descripción de cada propiedad.
Resultado: I

Se responden las mismas preguntas de prueba pero ahora con la función `responder_pregunta_embeddings` y se almacenan los resultados.


In [16]:
# Inicializar una lista vacía para almacenar las respuestas del chatbot de embeddings
resultados_embeddings = []

# Iterar a través de cada pregunta de prueba y obtener la respuesta del chatbot de embeddings
for pregunta in preguntas_prueba:
    respuesta = responder_pregunta_embeddings(pregunta)
    resultados_embeddings.append(respuesta)


Se itera a través de las preguntas de prueba, se obtienen las respuestas del chatbot embedding (Spacy) y las respuestas esperadas, se comparan y se imprimen los resultados de forma prolija (Correcto o Incorrecto)


In [17]:
# Imprimir los resultados del chatbot de Embeddings
print("\n--- Resultados del Chatbot de Embeddings ---")
for i in range(len(preguntas_prueba)):
    pregunta = preguntas_prueba[i]
    respuesta_embeddings = resultados_embeddings[i]
    respuesta_esperada = respuestas_esperadas[i]

    es_correcta = "Correcta" if respuesta_embeddings == respuesta_esperada else "Incorrecta"

    print(f"\nPregunta: {pregunta}")
    print(f"Respuesta Chatbot (Embeddings): {respuesta_embeddings}")
    print(f"Respuesta Esperada: {respuesta_esperada}")
    print(f"Resultado: {es_correcta}")


--- Resultados del Chatbot de Embeddings ---

Pregunta: Quiero buscar casas en venta.
Respuesta Chatbot (Embeddings): Puede buscar propiedades en venta utilizando nuestro buscador en la página principal. Puede filtrar por ubicación, tipo de propiedad, precio, etc.
Respuesta Esperada: Puede buscar propiedades en venta utilizando nuestro buscador en la página principal. Puede filtrar por ubicación, tipo de propiedad, precio, etc.
Resultado: Correcta

Pregunta: Qué necesito para alquilar un departamento?
Respuesta Chatbot (Embeddings): El tiempo promedio para vender una propiedad varía según el mercado, el precio y las características de la propiedad. Nuestros agentes pueden darle una estimación más precisa.
Respuesta Esperada: Los requisitos para alquilar varían según la propiedad y el propietario. Generalmente, se solicita comprobante de ingresos, garantía y referencias. Puede ver los requisitos específicos en la descripción de cada propiedad.
Resultado: Incorrecta

Pregunta: ¿Qué pape

## 5)

###Análisis y Conclusiones

Basado en los resultados de las pruebas, podemos comparar el rendimiento de los chatbots basados en TF-IDF y Embeddings:

### Chatbot TF-IDF

*   **Fortalezas:**
    *   Generalmente funciona bien con preguntas que contienen palabras clave exactas o muy similares a las del dataset de entrenamiento.
    *   Es relativamente simple de implementar y rápido para conjuntos de datos pequeños.
    *   Funciona correctamente en la mayoría de los casos de prueba presentados.

*   **Debilidades:**
    *   Puede tener dificultades para entender preguntas formuladas de manera diferente o que usan sinónimos que no están explícitamente en el dataset.
    *   Su comprensión se basa en la frecuencia de las palabras, no en el significado contextual. Esto se evidenció en la pregunta: "¿qué necesito para alquilar un departamento?", donde la respuesta fue incorrecta.


### Chatbot de Embeddings (SpaCy - es_core_news_md)

*   **Fortalezas:**
    *   Tiene una mejor capacidad para capturar el significado semántico de las preguntas, incluso si se usan palabras diferentes o estructuras de oración distintas.
    *   Puede manejar mejor la sinonimia y las variaciones en el lenguaje natural.

*   **Debilidades:**
    *   La precisión depende en gran medida de la calidad y el dominio del modelo de embedding pre-entrenado.
    *   Puede ser computacionalmente más costoso que TF-IDF, especialmente con datasets grandes.
    *   En este caso particular, también tuvo dificultades con algunas preguntas como "¿qué necesito para alquilar un departamento?" y "¿cómo pido una visita?", lo que sugiere que el modelo de embedding, aunque potente, puede no ser perfecto para el dominio específico de preguntas inmobiliarias sin un ajuste fino o un dataset de entrenamiento más grande y diverso.


### Comparación General

Ambos enfoques tienen sus méritos. TF-IDF es una línea base sólida y eficiente para tareas de recuperación simples, mientras que los embeddings ofrecen una comprensión semántica más profunda. Para un chatbot de producción, a menudo se utilizan enfoques híbridos o modelos más avanzados (como modelos basados en transformadores) que combinan lo mejor de ambos mundos o van más allá.

En este caso específico, ambos chatbots tuvieron un rendimiento similar en el conjunto de pruebas limitado. Sin embargo, el chatbot de embeddings tiene el potencial de generalizar mejor a preguntas no vistas si el modelo de embedding es adecuado para el dominio. Las fallas observadas en ambos chatbots resaltan la importancia de tener un dataset de preguntas y respuestas lo más completo y variado posible para cubrir las diferentes formas en que los usuarios pueden expresar la misma intención.

**Preguntas en las que fallan o tienen problemas:**

*   **TF-IDF:** "¿qué necesito para alquilar un departamento?" (respondió sobre documentos para vender)
*   **Embeddings:** "¿qué necesito para alquilar un departamento?" (respondió sobre el tiempo para vender) y "¿cómo pido una visita?" (respondió sobre hipotecas)


Ambos chatbots se confundieron con la pregunta relacionada con los requisitos para alquilar, lo que indica que la formulación de esta pregunta de prueba o la similitud con otras preguntas en el dataset de entrenamiento pudo haber sido un factor. El chatbot de Embeddings también se confundió con la pregunta sobre cómo pedir una visita.

**Conclusión:**

Si bien ambos métodos lograron responder correctamente a la mayoría de las preguntas de prueba, sus fallas en preguntas específicas demuestran la necesidad de refinar quizás el dataset para mejorar la precisión en un escenario real. Los embeddings tienen una ventaja teórica en la comprensión semántica, pero su rendimiento práctico depende en gran medida del modelo y los datos de entrenamiento.

*Fuentes:* Para este notebook se utilizó material de las clases, y la asistencia para escritura de código y análisis de Gemini, IA incorporada en Google Colab

#TP5

1) CONSIGNAS A RESOLVER

a) Creación del conjunto de datos de evaluación.

Además del dataset original que creó, debe crear un dataset de prueba o evaluación con la misma lógica: preguntas y respuestas.



In [18]:
preguntas_evaluacion = [
    "Todas las propiedades hipotecadas tienen seguro?",
    "Puedo vender mi casa aunque esté hipotecada",
    "El valor de las propiedades incluye todos los gastos?",
    "¿Ofrecen financiamiento?",
    "Tienen algo que esté amoblado?",
    "Necesito ayuda para eliminar del listado una propiedad",
    "Tengo problemas con el tema del alquiler",
    "Me pueden recomendar algún abogado o algún estudio jurídico?",
    "Cuánto se tarda en vender una casa aproximadamente?",
    "Cómo puedo saber si una casa registra deudas?"
]

respuestas_evaluacion = [
    "El seguro de hogar cubre daños a la propiedad y su contenido. En muchos casos, es obligatorio si tiene una hipoteca.",
    "Sí, es posible vender una propiedad con hipoteca. El proceso implica cancelar la hipoteca pendiente con los fondos de la venta.",
    "Al comprar una propiedad, debe considerar gastos adicionales como impuestos, honorarios notariales, gastos de registro, mudanza, etc.",
    "Podemos ponerlo en contacto con instituciones financieras que ofrecen opciones de financiamiento para la compra de propiedades.",
    "Sí, algunas de nuestras propiedades en alquiler están amuebladas. Puede usar los filtros de búsqueda para encontrar propiedades amuebladas.",
    "Para dar de baja su propiedad del listado, por favor contáctenos indicando los detalles de la propiedad y su solicitud.",
    "Si tiene un problema con su alquiler, le recomendamos contactar primero a su propietario o administrador. Si el problema persiste, podemos brindarle asesoramiento.",
    "No ofrecemos asesoramiento legal directo, pero podemos recomendarle abogados especializados en derecho inmobiliario.",
    "El tiempo promedio para vender una propiedad varía según el mercado, el precio y las características de la propiedad. Nuestros agentes pueden darle una estimación más precisa.",
    "Podemos ayudarlo a verificar si una propiedad tiene cargas (como hipotecas o embargos) solicitando un informe de dominio."
]

b) Debe elegir un modelo LLM de HuggingFace y al menos dos modelos de embeddings. Justifique su elección.


Decidí utilizar como modelos embeddings:
* intfloat/multilingual-e5-large
* BAAI/bge-m3

Y como modelo llm de HuggingFace el modelo:
* Qwen/Qwen2.5-1.5B-Instruct

Elegí Qwen 1.5B porque necesitaba un modelo liviano que no hiciera explotar la memoria de Colab, cosa que pasó porque al principio probé el 7B, y pués, saturé la memoria vram de colab gratuito (16gb).

Y para los embeddings, estoy comparando el BGE-M3 (que es más reciente) contra el E5-large (que es el estándar de la industria y más antiguo (2023)); para ver cuál de los dos recupera mejor la información técnica de los documentos de la inmobiliaria (tp4)

c) Implemente una clase ChatBot usando lo elegido en b).

Puede usar cualquier base de datos vectorial: Chroma y FAISS son las más documentadas. Recuerde que sus datos para su BD conocimiento es el dataset que Ud. planteó en el TP4.



d) Pruebe el chatbot creado en c) con las preguntas de su dataset a)

Usando los modelos elegidos en b), observe las respuestas generadas por el chatbot comparando al menos dos modelos de embeddings.

Justifique y determine cuál elegiría para su aplicación.


In [19]:
!pip install --upgrade pip
!pip install langchain langchain-huggingface langchain-community chromadb sentence-transformers transformers accelerate bitsandbytes datasets ragas openai --quiet


In [20]:
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [21]:
docs = [Document(page_content=f"Pregunta: {p}\nRespuesta: {r}", metadata={"idx": i})
        for i, (p, r) in enumerate(zip(preguntas_inmobiliaria, respuestas_inmobiliaria))]

print("Docs:", len(docs))


Docs: 25


In [22]:
def get_embedding_function(model_name):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Embedding model {model_name} -> device: {device}")
    model_kwargs = {"device": device}
    encode_kwargs = {"normalize_embeddings": True}
    return HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

EMB_MODELS = {
    "E5": "intfloat/multilingual-e5-large",
    "BGE": "BAAI/bge-m3"
}

embed_e5 = get_embedding_function(EMB_MODELS["E5"])
embed_bge = get_embedding_function(EMB_MODELS["BGE"])

Embedding model intfloat/multilingual-e5-large -> device: cuda


/tmp/ipython-input-435160398.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datas

Embedding model BAAI/bge-m3 -> device: cuda


In [23]:
def create_database(db_path, docs, embeddings):
    db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    db.add_documents(docs)
    db.persist()
    return db

db_e5 = create_database("chroma_db_e5", docs, embed_e5)
db_bge = create_database("chroma_db_bge", docs, embed_bge)

retriever_e5 = db_e5.as_retriever(search_kwargs={"k": 2})
retriever_bge = db_bge.as_retriever(search_kwargs={"k": 2})


/tmp/ipython-input-869442058.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  db = Chroma(persist_directory=db_path, embedding_function=embeddings)
/tmp/ipython-input-869442058.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [24]:
class ChatBot:
    def __init__(self, db, embedding_name="E5", top_k=1):
        # db: instancia Chroma
        self.db = db
        self.retriever = db.as_retriever(search_kwargs={"k": top_k})
        self.top_k = top_k

    def ask_with_contexts(self, question):
        # devuelve (respuesta_texto, docs[])
        docs = self.retriever.invoke(question)
        if not docs:
            return "No encontré información relevante.", []
        # por defecto devolvemos solo el documento más relevante concatenado (puede ajustarse)
        text = docs[0].page_content
        return text, docs

    def ask(self, question):
        text, _ = self.ask_with_contexts(question)
        # extraer solo la parte "Respuesta:" para mostrar corto
        if "Respuesta:" in text:
            return text.split("Respuesta:", 1)[1].strip()
        return text


In [25]:
chatbot_e5 = ChatBot(db_e5, embedding_name="E5", top_k=1)
chatbot_bge = ChatBot(db_bge, embedding_name="BGE", top_k=1)

Prueba unitaria de pregunta libre para testeo ⬇:

In [26]:
pregunta_test = "¿es malo vender con hipoteca?"
print("E5:", chatbot_e5.ask(pregunta_test))
print("BGE:", chatbot_bge.ask(pregunta_test))


E5: Sí, es posible vender una propiedad con hipoteca. El proceso implica cancelar la hipoteca pendiente con los fondos de la venta.
BGE: Sí, es posible vender una propiedad con hipoteca. El proceso implica cancelar la hipoteca pendiente con los fondos de la venta.


In [27]:
def evaluar_chatbots_dataframe(preguntas_eval, respuestas_eval, chatbot_e5, chatbot_bge):
    filas = []
    for pregunta, gt in zip(preguntas_eval, respuestas_eval):
        resp_e5 = chatbot_e5.ask(pregunta)
        resp_bge = chatbot_bge.ask(pregunta)
        filas.append({
            "pregunta": pregunta,
            "respuesta_real": gt,
            "respuesta_e5": resp_e5,
            "respuesta_bge": resp_bge
        })
    return pd.DataFrame(filas)

df_eval = evaluar_chatbots_dataframe(preguntas_evaluacion, respuestas_evaluacion, chatbot_e5, chatbot_bge)
df_eval


,pregunta,respuesta_real,respuesta_e5,respuesta_bge
0,Todas las propiedades hipotecadas tienen seguro?,El seguro de hogar cubre daños a la propiedad ...,El seguro de hogar cubre daños a la propiedad ...,El seguro de hogar cubre daños a la propiedad ...
1,Puedo vender mi casa aunque esté hipotecada,"Sí, es posible vender una propiedad con hipote...","Sí, es posible vender una propiedad con hipote...","Sí, es posible vender una propiedad con hipote..."
2,El valor de las propiedades incluye todos los ...,"Al comprar una propiedad, debe considerar gast...","Sí, todas nuestras propiedades listadas incluy...","Al comprar una propiedad, debe considerar gast..."
3,¿Ofrecen financiamiento?,Podemos ponerlo en contacto con instituciones ...,Podemos ponerlo en contacto con instituciones ...,Podemos ponerlo en contacto con instituciones ...
4,Tienen algo que esté amoblado?,"Sí, algunas de nuestras propiedades en alquile...","Sí, algunas de nuestras propiedades en alquile...","Sí, algunas de nuestras propiedades en alquile..."
5,Necesito ayuda para eliminar del listado una p...,"Para dar de baja su propiedad del listado, por...","Para dar de baja su propiedad del listado, por...","Para dar de baja su propiedad del listado, por..."
6,Tengo problemas con el tema del alquiler,"Si tiene un problema con su alquiler, le recom...","Si tiene un problema con su alquiler, le recom...","Si tiene un problema con su alquiler, le recom..."
7,Me pueden recomendar algún abogado o algún est...,"No ofrecemos asesoramiento legal directo, pero...","No ofrecemos asesoramiento legal directo, pero...","No ofrecemos asesoramiento legal directo, pero..."
8,Cuánto se tarda en vender una casa aproximadam...,El tiempo promedio para vender una propiedad v...,El tiempo promedio para vender una propiedad v...,El tiempo promedio para vender una propiedad v...
9,Cómo puedo saber si una casa registra deudas?,Podemos ayudarlo a verificar si una propiedad ...,Podemos ayudarlo a verificar si una propiedad ...,Podemos ayudarlo a verificar si una propiedad ...


Análisis General de Rendimiento:

Al revisar la tabla, se observa cómo ambos modelos de embedding, E5 y BGE, intentan responder a las preguntas de evaluación. Para cada pregunta, se presenta la respuesta real (ground truth) y las respuestas generadas por los chatbots E5 y BGE.

Ambos modelos de embedding demuestran la capacidad de recuperar la información correcta para varias preguntas (casi todas). Por ejemplo, para preguntas sobre la venta de propiedades hipotecadas, financiamiento o propiedades amuebladas, tanto E5 como BGE proporcionan la respuesta esperada.

* Discrepancias en E5: Para la pregunta "El valor de las propiedades incluye todos los gastos?", el modelo E5 respondió: "Sí, todas nuestras propiedades listadas incluyen fotos y, en muchos casos, videos o tours virtuales para que pueda tener una mejor idea de la propiedad.", lo cual es incorrecto. La respuesta real debería ser sobre gastos adicionales al comprar una propiedad. El modelo BGE, por su parte, demostró un mejor acoplamiento al contexto para la pregunta "El valor de las propiedades incluye todos los gastos?", respondiendo correctamente: "Al comprar una propiedad, debe considerar gastos adicionales como impuestos, honorarios notariales, gastos de registro, mudanza, etc.". Esto sugiere que BGE pudo capturar la intención de la pregunta de manera más efectiva en **este** caso específico.


e) BONUS (opcional)

Evalue el chatbot creado en c) para los modelos de embeddings elegidos y usados en d) usando el dataset a) con las métricas context precision y context recall. Puede probar usar la librería ragas o implementar Ud. mismo el cálculo de dichas métricas.

Dado sus resultados: ¿refuerza o no sus conclusiones realizadas en d) ?

Explique los resultados obtenidos y agregue sus conclusiones.



###A continuación, para entender mejor el desempeño de cada modelo, se obtendrán y analizarán las métricas "precisión" y "recall"

In [28]:
# Helper: obtener texto desde objetos Document u otros

def doc_to_text(obj):
    # Si es un Document (langchain_core.documents.Document), intentar leer .page_content
    try:
        content = obj.page_content
    except Exception:
        # si no tiene page_content, convertir a str
        content = str(obj)
    return content

# función robusta que computa embeddings para una lista de textos o documents
def compute_corpus_embeddings(docs_list, embedder):
    # producir lista de strings plana
    texts = [doc_to_text(d) for d in docs_list]

    # HuggingFaceEmbeddings (langchain_huggingface) suele exponer:
    #   - embed_documents(list[str])  -> lista de embeddings
    #   - embed_query(str) or embed_query([str]) -> embedding(s)
    # Usamos embed_documents si existe; si no, intentamos embed_query.
    if hasattr(embedder, "embed_documents"):
        embs = embedder.embed_documents(texts)
    elif hasattr(embedder, "embed_query"):
        # algunos wrappers aceptan list en embed_query
        try:
            # Try to embed the list of texts directly if the embedder supports it
            embs = embedder.embed_query(texts)
        except Exception:
            # Fallback: embed one by one if embed_query doesn't take a list directly
            embs = [embedder.embed_query(t) for t in texts]
            # embed_query can return arrays; ensure to convert to list of vectors
            embs = [e[0] if isinstance(e, (list, tuple, np.ndarray)) and len(e)==1 else e for e in embs]
    else:
        raise ValueError("Embedder no soporta embed_documents ni embed_query")
    return texts, np.array(embs)

# función que calcula precision/recall de contexto para 1 pregunta
# Solo tienes que reemplazar esta función
def context_precision_recall_for_question(question, respuesta_gt, retriever, embedder, corpus_texts, corpus_embs, k=2, threshold=0.65):
    # recuperar top-k docs
    docs = retriever.invoke(question)
    if docs is None:
        retrieved_texts = []
    else:
        retrieved_texts = [doc_to_text(d) for d in docs][:k]

    # embeddings: ground-truth
    if hasattr(embedder, "embed_query"):
        emb_gt = embedder.embed_query(respuesta_gt)
    else:
        emb_gt = embedder.embed_documents([respuesta_gt])[0]

    # retrieved embeddings
    if len(retrieved_texts) > 0:
        if hasattr(embedder, "embed_documents"):
            emb_ret = embedder.embed_documents(retrieved_texts)
        else:
            emb_ret = [embedder.embed_query(t) for t in retrieved_texts]
        emb_ret = np.array(emb_ret)
        sims_ret = cosine_similarity(emb_ret, [emb_gt]).flatten()
    else:
        sims_ret = np.array([])

    relevant_retrieved = int((sims_ret >= threshold).sum()) if sims_ret.size > 0 else 0
    precision = relevant_retrieved / max(k, 1)

    # calcular cuántos docs en el corpus son relevantes
    sims_corpus = cosine_similarity(corpus_embs, [emb_gt]).flatten()
    relevant_in_corpus = int((sims_corpus >= threshold).sum())

    # Si relevant_in_corpus es 0, el recall es 0.
    # Si relevant_retrieved > relevant_in_corpus (por duplicados), acotamos a 1.0
    if relevant_in_corpus == 0:
        recall = 0.0
    else:
        recall = min(relevant_retrieved / relevant_in_corpus, 1.0)

    return precision, recall, sims_ret
# Evaluar sobre todo el conjunto (E5 y BGE)
def evaluate_context_metrics(preguntas_eval, respuestas_eval, retriever, embedder, corpus_texts, corpus_embs, k=2, threshold=0.65):
    rows = []
    for q, gt in zip(preguntas_eval, respuestas_eval):
        prec, rec, sims = context_precision_recall_for_question(q, gt, retriever, embedder, corpus_texts, corpus_embs, k=k, threshold=threshold)
        rows.append({"pregunta": q, "gt": gt, "precision": prec, "recall": rec})
    return pd.DataFrame(rows)

# Ejecutar pre-cómputo de embeddings del corpus con E5 y BGE
# Usamos la lista `docs` original
texts_corpus_e5, corpus_embs_e5 = compute_corpus_embeddings(docs, embed_e5)
texts_corpus_bge, corpus_embs_bge = compute_corpus_embeddings(docs, embed_bge)

# Ejecutar evaluación
df_context_e5 = evaluate_context_metrics(preguntas_evaluacion, respuestas_evaluacion, retriever_e5, embed_e5, texts_corpus_e5, corpus_embs_e5, k=2, threshold=0.65)
df_context_e5["modelo"] = "E5"

df_context_bge = evaluate_context_metrics(preguntas_evaluacion, respuestas_evaluacion, retriever_bge, embed_bge, texts_corpus_bge, corpus_embs_bge, k=2, threshold=0.65)
df_context_bge["modelo"] = "BGE"

df_context = pd.concat([df_context_e5, df_context_bge], ignore_index=True)
df_context

,pregunta,gt,precision,recall,modelo
0,Todas las propiedades hipotecadas tienen seguro?,El seguro de hogar cubre daños a la propiedad ...,1.0,0.080000,E5
1,Puedo vender mi casa aunque esté hipotecada,"Sí, es posible vender una propiedad con hipote...",1.0,0.080000,E5
2,El valor de las propiedades incluye todos los ...,"Al comprar una propiedad, debe considerar gast...",1.0,0.080000,E5
3,¿Ofrecen financiamiento?,Podemos ponerlo en contacto con instituciones ...,1.0,0.080000,E5
4,Tienen algo que esté amoblado?,"Sí, algunas de nuestras propiedades en alquile...",1.0,0.080000,E5
5,Necesito ayuda para eliminar del listado una p...,"Para dar de baja su propiedad del listado, por...",1.0,0.080000,E5
6,Tengo problemas con el tema del alquiler,"Si tiene un problema con su alquiler, le recom...",1.0,0.080000,E5
7,Me pueden recomendar algún abogado o algún est...,"No ofrecemos asesoramiento legal directo, pero...",1.0,0.080000,E5
8,Cuánto se tarda en vender una casa aproximadam...,El tiempo promedio para vender una propiedad v...,1.0,0.080000,E5
9,Cómo puedo saber si una casa registra deudas?,Podemos ayudarlo a verificar si una propiedad ...,1.0,0.080000,E5


In [29]:
df_e5 = df_context[df_context['modelo'] == 'E5']

mean_precision_e5 = df_e5['precision'].mean()
mean_recall_e5 = df_e5['recall'].mean()

print(f"Average Precision for E5: {mean_precision_e5:.4f}")
print(f"Average Recall for E5: {mean_recall_e5:.4f}")

Average Precision for E5: 1.0000
Average Recall for E5: 0.0800


In [30]:
df_bge = df_context[df_context['modelo'] == 'BGE']

mean_precision_bge = df_bge['precision'].mean()
mean_recall_bge = df_bge['recall'].mean()

print(f"Average Precision for BGE: {mean_precision_bge:.4f}")
print(f"Average Recall for BGE: {mean_recall_bge:.4f}")

Average Precision for BGE: 1.0000
Average Recall for BGE: 0.9067


Ambos modelo embedding comparten una precisión promedio de 1.0 PERO, el promedio en Recall los separa por mucha distancia.

El modelo E5 alcanzó un Recall muy pobre, de tan sólo 0.08, mientras que el embedding BGE alcanzó un Recall muy bueno, de 0.9.


f) BONUS (opcional)

Evalue el modelo LLM usado para el chatbot creado en c) con el modelo de embedding que mejor le haya resultado en d) usando el dataset a).

Use las métricas Answer Relevancy  y Faithfulness. Puede probar usar la librería ragas o implementar Ud. mismo el cálculo de dichas métricas. Explique los resultados obtenidos y agregue sus conclusiones.



In [31]:
# Limpieza del contenido de documentos
def limpiar_doc(doc: Document):
    """
    Extrae solo la parte 'Respuesta:' del documento.
    """
    txt = doc.page_content
    if "Respuesta:" in txt:
        return txt.split("Respuesta:", 1)[1].strip()
    return txt.strip()

def docs_a_contexto(docs):
    """
    Convierte los documentos recuperados en un texto limpio y seguro para el prompt.
    Filtra Ellipsis, None, objetos raros, etc.
    """
    if docs is None:
        return ""

    # Si viene en formato raro (tuple u otro), forzar a lista
    if not isinstance(docs, (list, tuple)):
        docs = [docs]

    textos = []
    for d in docs:
        if d is None or d is Ellipsis:
            continue

        # Intentar extraer page_content
        contenido = ""
        if hasattr(d, "page_content"):
            contenido = d.page_content
        else:
            try:
                contenido = str(d)
            except Exception:
                continue

        # Evitar entradas tipo "Ellipsis" o vacías
        if contenido is None:
            continue
        if str(contenido).strip() in ["", "Ellipsis", "..."]:
            continue

        textos.append(str(contenido).replace("\n", " ").strip())

    return "\n".join(textos)

# --- Parser que limpia la salida del LLM ---
def limpiar_output(texto):
    texto = texto.replace("Assistant:", "").strip()
    return texto.split("\n\n")[0].strip()


class CleanParser(StrOutputParser):
    def parse(self, text):
        return limpiar_output(text)

parser = CleanParser()


In [32]:
MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True
)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    temperature=0.1
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)


`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Device set to use cuda:0
/tmp/ipython-input-2251539677.py:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [33]:
# Prompt

prompt_template = """
Usa SOLO la siguiente información para responder.
Si no es suficiente, responde: "No tengo información suficiente".

Información relevante:
{context}

Pregunta:
{question}

Respuesta breve y directa:
"""

prompt = PromptTemplate.from_template(prompt_template)



In [34]:
#Setup del retriever usando el mejor embedding
retriever = db_bge.as_retriever(search_kwargs={"k": 2})

In [35]:
# Sanitiza cualquier input antes de pasar al retriever
def sanitize_text(x):
    """
    Garantiza que el texto enviado al embedder NUNCA contenga objetos como Ellipsis.
    """
    if x is None:
        return ""
    if x is Ellipsis:
        return ""
    try:
        return str(x).replace("\n", " ").strip()
    except:
        return ""


In [36]:
def chain_rag(question):

    # Recuperación
    docs = retriever.invoke(question)
    contexto = docs_a_contexto(docs)

    # Construimos mensajes estilo chat
    messages = [
        {"role": "system", "content": "Eres un asistente experto en bienes raíces. Responde de forma breve y clara."},
        {
            "role": "user",
            "content": f"""Usa SOLO la siguiente información para responder.
Si no es suficiente, responde: "No tengo información suficiente".

Contexto:
{contexto}

Pregunta:
{question}"""
        }
    ]

    # Convertir al chat template de Qwen
    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # Tokenizar
    inputs = tokenizer(prompt_text, return_tensors="pt").to(model.device)

    # Generación
    outputs = model.generate(
        **inputs,
        max_new_tokens=180,
        temperature=0.1
    )

    # Decodificar
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Eliminar el prompt y quedarnos SOLO con la respuesta final
    respuesta_final = respuesta.split("<|im_start|>assistant")[-1].strip()

    return respuesta_final, contexto

In [37]:
preguntas_eval = preguntas_evaluacion         # lista de preguntas
respuestas_eval = respuestas_evaluacion        # ground truth

In [38]:
# Testing del LLM sobre todo el dataset de evaluación

respuestas_llm = []
contextos_llm = []

for q in preguntas_eval:
    resp, ctx = chain_rag(q)
    respuestas_llm.append(resp)
    contextos_llm.append(ctx)

df_llm = pd.DataFrame({
    "pregunta": preguntas_eval,
    "respuesta_gt": respuestas_eval,
    "respuesta_llm": respuestas_llm,
    "contexto_usado": contextos_llm
})

df_llm

,pregunta,respuesta_gt,respuesta_llm,contexto_usado
0,Todas las propiedades hipotecadas tienen seguro?,El seguro de hogar cubre daños a la propiedad ...,system\nEres un asistente experto en bienes ra...,Pregunta: qu es el seguro de hogar y es obliga...
1,Puedo vender mi casa aunque esté hipotecada,"Sí, es posible vender una propiedad con hipote...",system\nEres un asistente experto en bienes ra...,Pregunta: puedo vender mi propiedad si an teng...
2,El valor de las propiedades incluye todos los ...,"Al comprar una propiedad, debe considerar gast...",system\nEres un asistente experto en bienes ra...,Pregunta: qu gastos adicionales debo considera...
3,¿Ofrecen financiamiento?,Podemos ponerlo en contacto con instituciones ...,system\nEres un asistente experto en bienes ra...,Pregunta: ofrecen opciones de financiamiento R...
4,Tienen algo que esté amoblado?,"Sí, algunas de nuestras propiedades en alquile...",system\nEres un asistente experto en bienes ra...,Pregunta: puedo alquilar una propiedad amuebla...
5,Necesito ayuda para eliminar del listado una p...,"Para dar de baja su propiedad del listado, por...",system\nEres un asistente experto en bienes ra...,Pregunta: cmo puedo dar de baja mi propiedad d...
6,Tengo problemas con el tema del alquiler,"Si tiene un problema con su alquiler, le recom...",system\nEres un asistente experto en bienes ra...,Pregunta: qu hago si tengo un problema con mi ...
7,Me pueden recomendar algún abogado o algún est...,"No ofrecemos asesoramiento legal directo, pero...",system\nEres un asistente experto en bienes ra...,Pregunta: ofrecen asesoramiento legal Respuest...
8,Cuánto se tarda en vender una casa aproximadam...,El tiempo promedio para vender una propiedad v...,system\nEres un asistente experto en bienes ra...,Pregunta: cul es el tiempo promedio para vende...
9,Cómo puedo saber si una casa registra deudas?,Podemos ayudarlo a verificar si una propiedad ...,system\nEres un asistente experto en bienes ra...,Pregunta: cmo puedo saber si una propiedad est...


1.  Todas las propiedades hipotecadas tienen seguro? No tengo información suficiente.

2.  Puedo vender mi casa aunque esté hipotecada	Sí, es posible vender una propiedad con hipoteca. El proceso implica cancelar la hipoteca pendiente con los fondos de la venta.

3.  ¿Ofrecen financiamiento? Podemos ponerlo en contacto con instituciones financieras que ofrecen opciones de financiamiento para la compra de propiedades.

4.  Tienen algo que esté amoblado?  Sí, algunas de nuestras propiedades en alquiler están amuebladas. Para encontrar más detalles sobre las propiedades amuebladas disponibles, puede usar los filtros de búsqueda.

5.  Necesito ayuda para eliminar del listado una propiedad?. Para eliminar una propiedad del listado, por favor contacte con nosotros proporcionando los detalles de la propiedad y su solicitud.

6.  Tengo problemas con el tema del alquiler. Si tienes problemas con tu alquiler, te recomiendo que contactes primero con tu propietario o administrador. Si el problema persiste, podrías considerar nuestro servicio de asesoramiento para obtener ayuda profesional.

7.  Me pueden recomendar algún abogado o algún estudio jurídico?. No tengo información suficiente.

8.  Cuánto se tarda en vender una casa aproximadamente?. El tiempo promedio para vender una propiedad puede variar dependiendo del mercado, el precio y las características de la propiedad. Sin embargo, nuestro equipo de agentes puede proporcionar una estimación más precisa.

9.  Cómo puedo saber si una casa registra deudas?. Podemos ayudarlo a verificar si una propiedad tiene cargas como hipotecas o embargos solicitando un informe de dominio.

g) REFERENCIAS. Es obligatorio citar las fuentes de todo material utilizado para su TP incluido chats con IA generativa. Al final en su apartado Referencias liste todas las fuentes consultadas. Por ejemplo si mantuvo una conversación con ChatGPT puede agregar el enlace compartido a dicha conversación.



Referencias: